# CP Recommendation System


## Content Based filtering

First let's start with content based filtering
Here, we'll be concatenating the tags into a single string and run cosine similarity based on text to find the similarity between this particular problem and other problems


In [1]:
#importing stuff
import requests
import json
from pandas import json_normalize
import pandas as pd

#load data
tag_set = set()
data = requests.get("https://codeforces.com/api/problemset.problems").json()
problems_json = data["result"]["problems"]

problems = json_normalize(problems_json)
print(problems.isnull().sum())

contestId       0
index           0
name            0
type            0
points       2133
tags            0
rating        186
dtype: int64


As we can see, one of the columns has the name index which is overriding the original column "index", so let's rename it to something else.

In [2]:
column_list = list(problems.columns)
column_list[1] = "ID"
problems.columns = column_list
print(problems.isnull().sum())

contestId       0
ID              0
name            0
type            0
points       2133
tags            0
rating        186
dtype: int64


### Get Data From Users

Time to get the user dataset, we'll collect the data of all users who participated in atleast one contest.

In [3]:
def get_users():
    user_url = "https://codeforces.com/api/user.ratedList"
    user_data = requests.get(user_url).json()

    user_data = user_data["result"]

    df = json_normalize(user_data)
    return df

def get_users_from_csv():
    df = pd.read_csv("data/df_user.csv", encoding='utf-8')
    return df


The below code gets the submissions of one particular user.

In [53]:
def get_user_submissions(handle):
    start, count = (1, 999)

    user_url = "https://codeforces.com/api/user.status?handle={}&from={}&count={}"
    user_url = user_url.format(handle, start, count)
    user_data = requests.get(user_url).json()
    #if the response body is empty
    if user_data['status'] != 'OK':
        return pd.DataFrame()
    submissions = user_data["result"]
    df = json_normalize(submissions)
    return df

Now lets remove the users based on the following conditions
 - have rating less than or equal to 0
 - have stayed inactive for more than 1 year.

 ### Note: Run this below cell only if there is no csv file present, otherwise skip to the next one.

In [5]:
df_user = get_users()

In [6]:
from datetime import datetime, date, timedelta

df_user = df_user[['handle', 'country', 'rank', 'rating', 'maxRating', 'lastOnlineTimeSeconds']][df_user['rating']>0]

now = datetime.now()
lastYear = now.replace(year=now.year-1)

df_user["lastOnline"] = df_user['lastOnlineTimeSeconds'].map(lambda x: datetime.fromtimestamp(x))

df_user = df_user[df_user['lastOnline']  > lastYear][df_user.columns.difference(['lastOnlineTimeSeconds'], sort=False)]

df_user.to_csv("data/df_user.csv", index=False, encoding='utf-8')

Run this cell to load values into a dataframe from csv
### Note: Run this cell if you already have a df_user.csv with values stored.

In [70]:
df_user_total = get_users_from_csv()
df_user = df_user_total

In [66]:
def get_incorrect_user_submission(handle):
    user_submission = get_user_submissions(handle)
    return user_submission[user_submission["verdict"] != "OK"]["problem.name"]

def get_user_interactions(handle):
    user_submission = get_user_submissions(handle)
    if(len(user_submission.index) == 0):
        return
    problem_list = user_submission[user_submission["verdict"] != "OK"]["problem.name"].unique()
    for item in problem_list:
        yield [handle, item, 1]

def list_of_user_interactions():
    combined_list = []
    for user in df_user['handle']:
        for gen_item in get_user_interactions(user):
            combined_list.append(gen_item)
    return combined_list

In [ ]:
cols=['handle', 'problem_name', 'wrong_submission']
user_problem_interaction = pd.DataFrame(columns=cols)

for user in df_user['handle']:
    incorrect_user_submission = get_incorrect_user_submission(user)

    problem_freq = dict()
    for item in incorrect_user_submission:
        if item in problem_freq.keys():
            problem_freq[item] = problem_freq[item]+1
        else:
            problem_freq[item] = 1
    
    for item in problem_freq:
        new_row = [user, item, problem_freq[item]]
        df_new_row = pd.DataFrame([new_row], columns=cols)
        user_problem_interaction = pd.concat([user_problem_interaction, df_new_row])
        #user_problem_interaction.append({'handle': user, 'problem_name': item, 'wrong_submission': problem_freq[item]}, ignore_index=True)

user_problem_interaction = user_problem_interaction.reset_index(drop=True)
user_problem_interaction.to_csv("data/user_problem_interaction.csv", index=False, encoding='utf-8')

### Cosine Similarity
Now we are gonna start our project with content based filtering with cosine similarity

In [ ]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def combine_features(row):
    return row["type"]+" "+ " ".join(row["tags"])

problems["combined_features"] = problems.apply(combine_features, axis=1)

count_matrix = CountVectorizer().fit_transform(problems["combined_features"])
cosine_sim = cosine_similarity(count_matrix)

In [ ]:
def index_from_name(name):
    return problems[problems.name == name].index

problem_name = "Armchairs"
index_of_prob = index_from_name(problem_name).values[0]
tgs = problems.iloc[index_of_prob].tags

# print(problems[problems["tags"]])

### Neural Network

In [ ]:
df_user = df_user_total.sample(frac=0.01)

def batch_list_of_user_interactions(x):
    combined_list = []
    for user in df_user.iloc[x: x+50]['handle']:
        for gen_item in get_user_interactions(user):
            combined_list.append(gen_item)
    return combined_list

In [ ]:
# df_user_interaction = pd.DataFrame(list_of_user_interactions(), columns=['handle', 'problem', 'rating'])
# print(len(df_user_interaction))
combined_list = []
for i in range(0, 1, 50):
    combined_list.extend(list_of_user_interactions(i))
print(combined_list)